In [1]:
print('hello')

In [2]:
##### IMPORTS #####

import numpy as np

import pandas as pd

from functools import partial

import math

import operator

import itertools

##### IMPORTS #####

In [3]:
# Read the dataset
df = pd.read_csv("Notebook_Results/ENG-GER-FRE-N2-cleansed.csv", index_col='Unnamed: 0')
df = df.dropna()
df = df.reset_index(drop=True)
print(df); print(df.dtypes)

                      word  label
0                AACHENERS      2
1             AALÄHNLICHER      2
2             AALÄHNLICHES      2
3               AALARTIGES      2
4              AALBESTÄNDE      2
5               AALBEZOGEN      2
6             AALBEZOGENEN      2
7           AALBRUTLEITERN      2
8          AALBÜSCHELWELSE      2
9                AALBUTTEN      2
10               AALDORSCH      2
11                  AALEND      2
12                 AALENDE      2
13             AALENTNAHME      2
14               AALFÄNGER      2
15      AALFANGERGEBNISSES      2
16          AALFÄNGERINNEN      2
17         AALFANGMETHODEN      2
18          AALFANGVERBOTE      2
19         AALFANGVERBOTES      2
20          AALFANGVERBOTS      2
21              AALFISCHER      2
22            AALFISCHERIN      2
23             AALFLEISCHS      2
24              AALFÖRMIGE      2
25             AALFREUNDES      2
26                AALGLATT      2
27               AALGRÄTEN      2
28            

In [4]:
# Required functions for feature transformation

# Turn (uppercase) letters into numbers
def upper_to_better(letter):
    return ord(letter)-64 # ASCII-64: A=1,Z=sth,Specials=sth

# Turn words into arrays/lists/vectors
def word_2_vec2r_mk_36(word):
    return [upper_to_better(letter) for letter in str(word)]

# N-grams of letters(numbers) in each word(array)
def what_sup_my_ngram(arr, n):
    result = []
    for i in range(len(list(arr))-n+1):
        result.append(list(arr)[i:i + n])
    return result

# Count number of times a key appears in a list, basically same as python count for lists
def count_dracula(arr, key):
    return arr.count(key)

# Determine if a key appears in an array or not
def count_dracula_binary(arr, key):
    if arr.count(key) > 0:
        return 1
    return 0

In [5]:
# Acquire set of all unique characters --For 1-gram feature selection later

# Return set of all unique characters
def unique_letters(word, letter_bag):
    for c in word:
        if c not in letter_bag:
            letter_bag.add(c)
            
cleansed_letters = set()
df['word'].apply(unique_letters,args=[cleansed_letters])
print("Cleansed letters \n%s \nof length %s" % (list(cleansed_letters), len(cleansed_letters)))

Cleansed letters 
['Á', 'Ã', 'Û', 'U', 'Ë', 'D', 'G', 'Ö', 'Í', 'Î', 'É', 'Ù', 'Ú', 'Ñ', 'Y', 'X', 'Ç', 'Ï', 'E', 'È', 'W', 'M', 'J', 'P', 'Ø', 'L', 'K', 'Q', 'Ä', 'Å', 'Ô', 'C', 'Â', 'B', 'Z', 'R', 'N', 'I', 'F', 'Ê', 'S', 'V', 'T', 'A', 'H', 'O', 'Ó', 'Ü'] 
of length 48


In [6]:
# Transform word column into array/vectors
df['word'] = df['word'].apply(word_2_vec2r_mk_36)
# print(df); print(df.dtypes)

In [7]:
# Create new column of word_length
df['word_length'] = df['word'].apply(len)
# print(df); print(df.dtypes)

In [45]:
# Create lists of 1-2-3-grams of each word as 3 columns
df["word_1_gram"] = df['word'].apply(what_sup_my_ngram, args=[1])
df["word_2_gram"] = df['word'].apply(what_sup_my_ngram, args=[2])
df["word_3_gram"] = df['word'].apply(what_sup_my_ngram, args=[3])

In [9]:
# Required functions for feature selection

# MUTUAL ~REDONE~ REBORN
def mutual_information(dataframe, feature_list, label, num_of_feats, non_featlabel_col):
    left_feats = feature_list.copy()
    label_grouped = dataframe.groupby(label).count()
    label_values = dataframe[label].unique()
    label_sum = sum(label_grouped[non_featlabel_col])
    label_probs = dict.fromkeys(label_values, 0)
    for label_val in label_values:
        label_probs[label_val] = label_grouped[non_featlabel_col][label_val]/label_sum
    informative_feats = []
    informative_feats_str = []
    informative_feat_probs = []
    for i in range(num_of_feats):
        print("%s %s %s %s %s %s %s" %(i, i, i, i, i, i, i))
        current_feats = []
        current_feats_str = []
        current_feats_probs = []
        feat_dict = dict.fromkeys(left_feats, 0)
        for current_feat in left_feats:
            current_feats = informative_feats.copy()
            current_feats_str = informative_feats_str.copy()
            current_feats_probs = informative_feat_probs.copy()
            current_feats.append(df[current_feat].unique())
            current_feats_str.append(current_feat)
            # informative_feats.append(df[current_feat].unique())
            # informative_feats_str.append(current_feat)
            # grouped = dataframe.pivot_table(index=label, columns= current_feats_str, values=non_featlabel_col, fill_value=0, aggfunc='count').unstack().to_frame().rename(columns={0: non_featlabel_col})
            feat_grouped = dataframe.groupby(current_feat).count()
            feat_values = dataframe[current_feat].unique()
            feat_probs = dict.fromkeys(feat_values, 0)
            for feat_val in feat_values:
                feat_probs[feat_val] = feat_grouped[non_featlabel_col][feat_val]/label_sum
            current_feats_probs.append(feat_probs)
            current_feats.append(df[label].unique())
            current_feats_str.append(label)
            current_feats_probs.append(label_probs)
            grouped = dataframe.groupby(current_feats_str).count()
            levels = []
            for j in range(len(current_feats_str)):
                levels.append(grouped.index.levels[j].values)
            # levels = [grouped.index.levels[0].values, grouped.index.levels[1].values, list(range(1, 4))]
            new_index = pd.MultiIndex.from_product(levels, names=grouped.index.names)
            grouped = grouped.reindex(new_index, fill_value=0)
            # print("%s %s %s %s %s %s %s" % (current_feats_str, current_feats_str, current_feats_str, current_feats_str, current_feats_str, current_feats_str, current_feats_str))
            # print(current_feats)
            # print(current_feats_probs)
            info = 0
            for item in list(itertools.product(*current_feats)):
                item = list(item)
                # print(item)
                # print(grouped)
                intersection_count = grouped[non_featlabel_col]
                multiplication = 1
                for j in range(len(item)):
                    # print("j = %s, int = %s" %(j, intersection_count))
                    intersection_count = intersection_count[item[j]]
                    multiplication = multiplication * current_feats_probs[j][item[j]]
                if (intersection_count != 0):
                    intersection_count = intersection_count/label_sum
                    info = info + intersection_count*math.log(intersection_count/multiplication)
            # for feat in informative_feats:
            #     feat_values = dataframe[feat].unique()
            #     for feat_val in feat_values:
            #         for label_val in label_values:
            #             info = info + 5
            feat_dict[current_feat] = info
            # informative_feats.remove(df[current_feat].unique())
            # informative_feats_str.remove(current_feat)
            # current_feats = []
            # current_feats_str = []
            # current_feats_probs = []
        top_feat = max(feat_dict, key=feat_dict.get)
        informative_feats.append(df[top_feat].unique())
        informative_feats_str.append(top_feat)
        top_feat_grouped = dataframe.groupby(top_feat).count()
        top_feat_values = dataframe[top_feat].unique()
        top_feat_probs = dict.fromkeys(top_feat_values, 0)
        for feat_val in top_feat_values:
            top_feat_probs[feat_val] = top_feat_grouped[non_featlabel_col][feat_val] / label_sum
        informative_feat_probs.append(top_feat_probs)
        left_feats.remove(top_feat)
    return informative_feats_str

In [10]:
# 1-gram feature extraction & selection

In [31]:
# Make 2 seperate columns for each letter(1-gram): for count and binary appearance

for letter in cleansed_letters:
    # name = "word_%s_gram" % (letter)
    # df[name] = df["word_1+_gram"].count(ord(letter)-64)
    # df["word_%s_gram" % (letter)] = df["word_1_gram"].apply(count_dracula, args=[[ord(letter)-64]])
    df["word_%s_gram_bin" % (letter)] = df["word_1_gram"].apply(count_dracula_binary, args=[[upper_to_better(letter)]])
    df["word_%s_gram" % (letter)] = df["word_1_gram"].apply(count_dracula, args=[[upper_to_better(letter)]])

In [13]:
# Select top 10 most informative 1-gram features

cleansed_columns = ['word_{0}_gram_bin'.format(element) for element in cleansed_letters]
informative_letters = mutual_information(df, cleansed_columns, 'label', 10, 'word')
informative_letters = [element[5] for element in informative_letters]
print("informative letters are %s" % (informative_letters))
# informative_letters are ['É', 'K', 'E', 'H', 'C', 'G', 'N', 'Y', 'Ä', 'S']

0 0 0 0 0 0 0
1 1 1 1 1 1 1
2 2 2 2 2 2 2
3 3 3 3 3 3 3
4 4 4 4 4 4 4
5 5 5 5 5 5 5
6 6 6 6 6 6 6
7 7 7 7 7 7 7
8 8 8 8 8 8 8
9 9 9 9 9 9 9
informative letters are ['É', 'K', 'E', 'H', 'C', 'G', 'N', 'Y', 'Ä', 'S']


In [14]:
# 2-gram feature extraction & selection

In [15]:
# Select the set of top most frequent 1-gram features 
# (In order to create limited amount of 2-gram features)

countOf1grams = {}
for letter in cleansed_letters:
    countOf1grams[letter] = df["word_%s_gram" % (letter)].sum()
sorted_countOf1grams = sorted(countOf1grams.items(), key=operator.itemgetter(1), reverse=True) # is a list now
# print(sorted_countOf1grams)
most_freq_20_1grams = sorted_countOf1grams[:20]
# print(len(sorted_countOf1grams))
print(most_freq_20_1grams)
# print(len(most_freq_20_1grams))

twograms_from_top20 = set()
# firstly, top10 + top10_and_middle10
for i in range(5):
    for j in range(10):
        char1 = most_freq_20_1grams[i][0]
        char2 = most_freq_20_1grams[j][0]
        string1 = char1 + char2
        string2 = char2 + char1
        twograms_from_top20.add( string1)
        twograms_from_top20.add( string2)
print(twograms_from_top20)

[('E', 1146635), ('N', 722320), ('S', 686704), ('R', 671476), ('I', 666232), ('A', 636838), ('T', 612459), ('O', 437102), ('L', 435611), ('U', 347674), ('C', 312833), ('H', 283172), ('G', 261529), ('M', 241284), ('D', 240320), ('P', 214625), ('B', 184331), ('F', 151400), ('K', 127002), ('Y', 81945)]
{'AN', 'LS', 'UN', 'NI', 'ES', 'RE', 'TE', 'SN', 'NU', 'US', 'RN', 'NO', 'UR', 'EI', 'IA', 'NT', 'LN', 'EA', 'RO', 'IS', 'EU', 'TS', 'IU', 'TR', 'IL', 'OE', 'RA', 'NA', 'NR', 'NL', 'ET', 'SI', 'SO', 'ST', 'RL', 'RU', 'IT', 'SE', 'UI', 'LI', 'SU', 'ER', 'IE', 'RT', 'AR', 'LR', 'SR', 'NN', 'ON', 'NS', 'NE', 'UE', 'IN', 'SS', 'RR', 'AE', 'RS', 'IR', 'OR', 'IO', 'EE', 'RI', 'OI', 'EL', 'SA', 'TN', 'EN', 'EO', 'LE', 'AS', 'AI', 'OS', 'SL', 'II', 'TI'}


In [32]:
# Get rid of unnecessary 1-gram columns

for letter in cleansed_letters-set(informative_letters):
    df = df.drop("word_%s_gram" % (letter), 1)
    df = df.drop("word_%s_gram_bin" % (letter), 1)
for letter in informative_letters:
    df = df.drop("word_%s_gram" % (letter), 1)
# print(df); print(df.dtypes)

In [33]:
# Make 2 seperate columns for each 2-gram: for count and binary appearance

for twoGram in twograms_from_top20:
    df["word_%s_gram_bin" % (twoGram)] = df["word_2_gram"].apply(count_dracula_binary, args=[[upper_to_better(twoGram[0]), upper_to_better(twoGram[1])]])
    df["word_%s_gram" % (twoGram)] = df["word_2_gram"].apply(count_dracula, args=[[upper_to_better(twoGram[0]), upper_to_better(twoGram[1])]])

In [20]:
# Select top 5 most informative 2-gram features

cleansed_2gram_names = ['word_{0}_gram_bin'.format(element) for element in twograms_from_top20]
informative_2grams = mutual_information(df, cleansed_2gram_names, 'label', 5, 'word')
informative_2grams = [element[5:7] for element in informative_2grams]
print("informative 2grams are %s" % (informative_2grams))
# informative 2grams are ['EN', 'EI', 'NT', 'ER', 'TE']

0 0 0 0 0 0 0
1 1 1 1 1 1 1
2 2 2 2 2 2 2
3 3 3 3 3 3 3
4 4 4 4 4 4 4
informative 2grams are ['EN', 'EI', 'NT', 'ER', 'TE']


In [21]:
# 3-gram feature extraction & selection

In [22]:
# Select the set of top most frequent 2-gram features 
# (In order to create limited amount of 3-gram features)

countOf2grams = {}
for twogram in twograms_from_top20:
    countOf2grams[twogram] = df["word_%s_gram" % (twogram)].sum()
sorted_countOf2grams = sorted(countOf2grams.items(), key=operator.itemgetter(1), reverse=True)  # is a list now
# print(sorted_countOf2grams)
most_freq_20_2grams = sorted_countOf2grams[:20]
# print(len(sorted_countOf2grams))
print(most_freq_20_2grams)
# print(len(most_freq_20_2grams))

threegrams_from_top = set()
# firstly, top20*2gram + top10*1gram
for i in range(5):
    for j in range(10):
        char1 = most_freq_20_1grams[i][0]
        char2 = most_freq_20_2grams[j][0]
        string1 = char1 + char2
        string2 = char2 + char1
        threegrams_from_top.add( string1)
        threegrams_from_top.add( string2)
print(threegrams_from_top)

[('ER', 232921), ('EN', 220561), ('TE', 153348), ('ES', 119237), ('IN', 117297), ('RE', 108444), ('ST', 105408), ('AN', 94773), ('TI', 90319), ('ON', 90055), ('RA', 87203), ('UN', 85971), ('IS', 81621), ('LE', 80980), ('NT', 80832), ('RI', 72235), ('NE', 68541), ('SE', 67386), ('EI', 66518), ('IE', 65827)]
{'RTI', 'RIN', 'INI', 'NTI', 'ENI', 'ONN', 'SRE', 'ANS', 'STI', 'TIN', 'STS', 'ONS', 'ANE', 'ANN', 'ESS', 'SEN', 'IER', 'EES', 'STR', 'ENN', 'TIR', 'EAN', 'ERN', 'ERS', 'RTE', 'SIN', 'NON', 'EIN', 'INN', 'TER', 'SAN', 'ESR', 'NER', 'SER', 'ONR', 'NTE', 'ENS', 'INE', 'ANR', 'SON', 'RST', 'IEN', 'REN', 'REE', 'STE', 'ERR', 'ITI', 'ESI', 'NIN', 'TIS', 'STN', 'ETI', 'SES', 'EON', 'ETE', 'ANI', 'EEN', 'ESN', 'NEN', 'RON', 'EST', 'IIN', 'IAN', 'TES', 'TII', 'ENR', 'ENE', 'NES', 'ERE', 'INS', 'REI', 'ION', 'IES', 'TEE', 'TEI', 'TEN', 'ONE', 'IST', 'TIE', 'ESE', 'ONI', 'INR', 'SST', 'RAN', 'NST', 'RRE', 'EER', 'NAN', 'IRE', 'ITE', 'RES', 'ERI', 'RER', 'NRE'}


In [36]:
# Get rid of unnecessary 2-gram columns

for twogram in twograms_from_top20-set(informative_2grams):
    df = df.drop("word_%s_gram" % (twogram), 1)
    df = df.drop("word_%s_gram_bin" % (twogram), 1)
for twogram in informative_2grams:
    df = df.drop("word_%s_gram" % (twogram), 1)
# print(df); print(df.dtypes)

In [46]:
# Make 2 seperate columns for each 3-gram: for count and binary appearance

for threeGram in threegrams_from_top:
    df["word_%s_gram_bin" % (threeGram)] = df["word_3_gram"].apply(count_dracula_binary, args=[[upper_to_better(threeGram[0]), upper_to_better(threeGram[1]), upper_to_better(threeGram[2])]])
    df["word_%s_gram" % (threeGram)] = df["word_3_gram"].apply(count_dracula, args=[[upper_to_better(threeGram[0]), upper_to_better(threeGram[1]), upper_to_better(threeGram[2])]])

In [27]:
# Select top 5 most informative 3-gram features

cleansed_3gram_names = ['word_{0}_gram_bin'.format(element) for element in threegrams_from_top]
informative_3grams = mutual_information(df, cleansed_3gram_names, 'label', 5, 'word')
informative_3grams = [element[5:8] for element in informative_3grams]
print("informative 3grams are %s" % (informative_3grams))
# informative 3grams are ['TEN', 'STE', 'TER', 'NTE', 'ERN']

0 0 0 0 0 0 0
1 1 1 1 1 1 1
2 2 2 2 2 2 2
3 3 3 3 3 3 3
4 4 4 4 4 4 4
informative 3grams are ['TEN', 'STE', 'TER', 'NTE', 'ERN']


In [47]:
# Get rid of unnecessary 3-gram columns

for threegram in threegrams_from_top-set(informative_3grams):
    df = df.drop("word_%s_gram" % (threegram), 1)
    df = df.drop("word_%s_gram_bin" % (threegram), 1)
for threegram in informative_3grams:
    df = df.drop("word_%s_gram" % (threegram), 1)
print(df); print(df.dtypes)

                                                     word  label  word_length  \
0                          [1, 1, 3, 8, 5, 14, 5, 18, 19]      2            9   
1              [1, 1, 12, 132, 8, 14, 12, 9, 3, 8, 5, 18]      2           12   
2              [1, 1, 12, 132, 8, 14, 12, 9, 3, 8, 5, 19]      2           12   
3                      [1, 1, 12, 1, 18, 20, 9, 7, 5, 19]      2           10   
4                 [1, 1, 12, 2, 5, 19, 20, 132, 14, 4, 5]      2           11   
5                      [1, 1, 12, 2, 5, 26, 15, 7, 5, 14]      2           10   
6               [1, 1, 12, 2, 5, 26, 15, 7, 5, 14, 5, 14]      2           12   
7       [1, 1, 12, 2, 18, 21, 20, 12, 5, 9, 20, 5, 18,...      2           14   
8       [1, 1, 12, 2, 156, 19, 3, 8, 5, 12, 23, 5, 12,...      2           15   
9                        [1, 1, 12, 2, 21, 20, 20, 5, 14]      2            9   
10                        [1, 1, 12, 4, 15, 18, 19, 3, 8]      2            9   
11                          

In [48]:
# Get rid of 'list of 1-2-3 grams in word' column
df = df.drop('word_1_gram', 1)
df = df.drop('word_2_gram', 1)
df = df.drop('word_3_gram', 1)

In [49]:
print(df); print(df.dtypes)

                                                     word  label  word_length  \
0                          [1, 1, 3, 8, 5, 14, 5, 18, 19]      2            9   
1              [1, 1, 12, 132, 8, 14, 12, 9, 3, 8, 5, 18]      2           12   
2              [1, 1, 12, 132, 8, 14, 12, 9, 3, 8, 5, 19]      2           12   
3                      [1, 1, 12, 1, 18, 20, 9, 7, 5, 19]      2           10   
4                 [1, 1, 12, 2, 5, 19, 20, 132, 14, 4, 5]      2           11   
5                      [1, 1, 12, 2, 5, 26, 15, 7, 5, 14]      2           10   
6               [1, 1, 12, 2, 5, 26, 15, 7, 5, 14, 5, 14]      2           12   
7       [1, 1, 12, 2, 18, 21, 20, 12, 5, 9, 20, 5, 18,...      2           14   
8       [1, 1, 12, 2, 156, 19, 3, 8, 5, 12, 23, 5, 12,...      2           15   
9                        [1, 1, 12, 2, 21, 20, 20, 5, 14]      2            9   
10                        [1, 1, 12, 4, 15, 18, 19, 3, 8]      2            9   
11                          

In [50]:
# Save to a csv file
df.to_csv("Notebook_Results/ENG-GER-FRE-N3-bin-informative.csv")

In [3]:
# Re-load --just to prevent rerunning previous cells
df = pd.read_csv("ENG-GER-FRE-N3-bin-informative.csv", index_col='Unnamed: 0')
# df = df.dropna()
# df = df.reset_index(drop=True)
# print(df); print(df.dtypes)

                                                     word  label  word_length  \
0                          [1, 1, 3, 8, 5, 14, 5, 18, 19]      2            9   
1              [1, 1, 12, 132, 8, 14, 12, 9, 3, 8, 5, 18]      2           12   
2              [1, 1, 12, 132, 8, 14, 12, 9, 3, 8, 5, 19]      2           12   
3                      [1, 1, 12, 1, 18, 20, 9, 7, 5, 19]      2           10   
4                 [1, 1, 12, 2, 5, 19, 20, 132, 14, 4, 5]      2           11   
5                      [1, 1, 12, 2, 5, 26, 15, 7, 5, 14]      2           10   
6               [1, 1, 12, 2, 5, 26, 15, 7, 5, 14, 5, 14]      2           12   
7       [1, 1, 12, 2, 18, 21, 20, 12, 5, 9, 20, 5, 18,...      2           14   
8       [1, 1, 12, 2, 156, 19, 3, 8, 5, 12, 23, 5, 12,...      2           15   
9                        [1, 1, 12, 2, 21, 20, 20, 5, 14]      2            9   
10                        [1, 1, 12, 4, 15, 18, 19, 3, 8]      2            9   
11                          

In [7]:
# Test and Train
mask = np.random.rand(len(df)) < 0.8
df_train = df[mask]
df_train = df_train.reset_index(drop=True)
df_test = df[~mask]
df_test = df_test.reset_index(drop=True)
# print('df_train')
# print(df_train); print(df_train.dtypes)
# print('df_test')
# print(df_test); print(df_test.dtypes)

df_train
                                                     word  label  word_length  \
0              [1, 1, 12, 132, 8, 14, 12, 9, 3, 8, 5, 18]      2           12   
1              [1, 1, 12, 132, 8, 14, 12, 9, 3, 8, 5, 19]      2           12   
2                      [1, 1, 12, 1, 18, 20, 9, 7, 5, 19]      2           10   
3                 [1, 1, 12, 2, 5, 19, 20, 132, 14, 4, 5]      2           11   
4                      [1, 1, 12, 2, 5, 26, 15, 7, 5, 14]      2           10   
5               [1, 1, 12, 2, 5, 26, 15, 7, 5, 14, 5, 14]      2           12   
6       [1, 1, 12, 2, 18, 21, 20, 12, 5, 9, 20, 5, 18,...      2           14   
7       [1, 1, 12, 2, 156, 19, 3, 8, 5, 12, 23, 5, 12,...      2           15   
8                        [1, 1, 12, 2, 21, 20, 20, 5, 14]      2            9   
9                         [1, 1, 12, 4, 15, 18, 19, 3, 8]      2            9   
10                                   [1, 1, 12, 5, 14, 4]      2            6   
11                 

In [8]:
# Save test/train to files
df_train.to_csv("Notebook_Results/TRAIN-ENG-GER-FRE-N3-bin-informative.csv")
df_test.to_csv("Notebook_Results/TEST-ENG-GER-FRE-N3-bin-informative.csv")